In [19]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools

%matplotlib inline

In [23]:
# Read data from .csv files 

loc = "voetbal_1956-20220309/"

raw_cards_file = pd.read_csv (loc + '_cards.csv')
raw_goals_file = pd.read_csv (loc + '_goals.csv')
raw_matches_file = pd.read_csv (loc + '_matches.csv')
raw_players_file = pd.read_csv (loc + '_players.csv')
raw_subs_file = pd.read_csv (loc + '_subs.csv')

In [31]:
raw_cards_file['round'] = pd.to_numeric(raw_cards_file['round'])
raw_cards_file.isnull()

,Unnamed: 0,saison,round,home,away,team,player,minute,type
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...
32287,False,False,False,False,False,False,False,False,False
32288,False,False,False,False,False,False,False,False,False
32289,False,False,False,False,False,False,False,False,False
32290,False,False,False,False,False,False,False,False,False
